# ReMi-DAS: Refraction Microtremor Processing for DAS

This Jupyter notebook demonstrates the application of the **ReMi-DAS** workflow for shear-wave velocity profiling using DAS data. The notebook guides users through a series of processing steps adapted from conventional Refraction Microtremor (ReMi) techniques, including:

- Reading and preprocessing DAS data
- Slowness-frequency (p-f) transformation
- Extraction of Rayleigh wave dispersion curves

The ReMi-DAS implementation uses the functionality of the [**DASCore**](https://github.com/DASDAE/dascore) package for signal processing.

### References

- McMechan, G.A. and Yedlin, M.J., 1981. Analysis of dispersive waves by wave field transformation. *Geophysics*, 46(6), pp.869-874.  
- Louie, J.N., 2001. Faster, better: shear-wave velocity to 100 meters depth from refraction microtremor arrays. *BSSA*, 91(2), pp.347-364. 
- Chambers, D., Jin, G., Tourei, A., Issah, A.H.S., Lellouch, A., Martin, E.R., Zhu, D., Girard, A.J., Yuan, S., Cullison, T. and Snyder, T., 2024. Dascore: A python library for distributed fiber optic sensing. *Seismica*, 3(2), pp.10-26443.


In [ ]:
# Packages installation if not already installed

# !pip install dascore --upgrade --quiet
# !pip install PyQt5 --upgrade --quiet

In [1]:
import os
import numpy as np
from scipy.interpolate import CubicSpline

import dascore as dc
from dascore.units import Hz

from matplotlib import pyplot as plt

# Use Qt backend for interactive plots
%matplotlib qt

### Reading and Preprocessing DAS Data

*Note: You can skip this section if you're using the provided example dataset (`example.h5`).*

In [ ]:
# Define file path and filename for DAS data
folder_path = 'Path/To/Your/DAS/Folder'  # Replace with your actual folder path
filename = 'data.hdf5'  # Replace with your actual DAS file name

full_path = os.path.join(folder_path, filename)

# Read and preprocess data
pa = dc.spool(full_path)
pa_fil = (
    pa[0]
    .set_units("1/s", distance="m", time="s")  # Set physical units
    .detrend("time")                           # Remove linear trend
    .decimate(time=20)                         # Downsample in time
    .taper(time=0.05)                          # Apply taper in time
    .pass_filter(time=(1 * Hz, 49 * Hz))       # Bandpass filter 
    .select(distance=(102, 222), samples=True) # Select a distance range
    .transpose('distance', 'time')             
)

### Using the Example Preprocessed Data

The file `example.h5` contains data that has already been preprocessed using the steps described above.


In [2]:
# Read the preprocessed example data and visualize it
pa_fil = dc.spool('./example.h5')

# Plot the preprocessed data
fig, ax = plt.subplots(figsize=(7, 5))
pa_fil[0].viz.waterfall(ax=ax,scale=0.0001)

<Axes: xlabel='time', ylabel='distance(m)'>

### Velocity Spectral (p-f) Analysis

This section performs slowness-frequency transformation on the DAS data.  
The workflow includes:
- Chunking the data into overlapping time windows.
- Applying tapers in both time and distance to reduce edge effects.
- Performing a tau-p (slant stack) transform to map the data into the slowness domain.
- Computing the discrete Fourier transform (DFT) along the time axis to obtain the frequency content for each slowness.
- The resulting spectra are used for subsequent stacking, normalization, and dispersion curve picking.

In [3]:
# Define processing functions for each step
def taper(patch, time, window_type):
    """Apply a taper along the time axis."""
    return patch.taper(time=0.1, window_type=window_type)

def taper_d(patch, distance, window_type):
    """Apply a taper along the distance axis."""
    return patch.taper(distance=0.1, window_type=window_type)

def taup(patch, velocities):
    """Apply tau-p (slant stack) transform with given velocities."""
    return patch.tau_p(velocities)

def dft(patch, dim="time"):
    """Apply discrete Fourier transform along the specified dimension."""
    return patch.dft(dim="time", real=True)


# Process the data step by step
sp_fil = dc.spool(pa_fil)

# Chunk the data into segments of 30 seconds with 10 seconds overlap
sp_fil_chunked = sp_fil.chunk(time=30, overlap=10)

# Apply tapering in time
sp_fil_chunked_taper = dc.spool(
    sp_fil_chunked.map(taper, time=0.1, window_type="hann")
)

# Apply tapering in distance
sp_fil_chunked_taper_d = dc.spool(
    sp_fil_chunked_taper.map(taper_d, distance=0.1, window_type="hann")
)

# Apply tau-p transform with specified velocities
sp_fil_chunked_taper_taup = dc.spool(
    sp_fil_chunked_taper_d.map(taup, velocities=np.arange(100, 800, 20))
)

# Apply discrete Fourier transform along the time dimension
sp_fil_chunked_taper_taup_dft = dc.spool(
    sp_fil_chunked_taper_taup.map(dft, dim="time")
)

Output()

Output()

Output()

Output()

Output()

Output()

Output()

Output()

### Spectral normalization and stacking

This section computes the normalized and stacked power spectrum across all data chunks.  
For each chunk, the code:
- Calculates the power spectrum (magnitude squared of the DFT).
- Splits the spectrum into positive and negative slowness components and sums them to enforce symmetry.
- Normalizes each frequency by the average amplitude across slowness.
- Stacks (sums) the normalized spectra from all chunks to enhance coherent features and suppress noise.

The result, `summed_avg_stack`, is a 2D array representing the normalized, stacked power as a function of frequency and slowness, ready for visualization and dispersion analysis.

In [13]:
# Calculate number of frequency bins and midpoint
n_freq = sp_fil_chunked_taper_taup_dft[0].data.shape[0]
half = n_freq // 2

# Initialize the stack for the averaged, normalized spectra
if n_freq % 2 == 0:
    summed_avg_stack = np.zeros((half, sp_fil_chunked_taper_taup_dft[0].data.shape[1]))
else:
    summed_avg_stack = np.zeros((half + 1, sp_fil_chunked_taper_taup_dft[0].data.shape[1]))

# Loop through each chunked patch and sum normalized power spectra
for i, patch in enumerate(sp_fil_chunked_taper_taup_dft):
    
    # Compute power spectrum 
    power_spectrum = (patch.data * np.conj(patch.data)).real

    if n_freq % 2 == 0:
        # Split into negative and positive slowness components
        # Even: includes Nyquist frequency
        neg = power_spectrum[:half,:]
        pos = power_spectrum[half:,:]
        neg_flipped  = np.flipud(neg)
        summed = pos + neg_flipped
    else:
        # Split into negative and positive slowness components
        # Odd
        pos = power_spectrum[:half+1]
        neg = power_spectrum[half+1:]
        neg_flipped  = np.flipud(neg)
        summed = pos + neg_flipped[:pos.shape[0]]

    # Normalize by the average across slowness for each frequency
    avg = np.sum(summed, axis=0) / len(summed)
    summed_avg = summed/avg

    # Accumulate the normalized spectra
    summed_avg_stack = summed_avg_stack + summed_avg

### Rayleigh Phase-Velocity Dispersion Picking

This section enables interactive picking of the Rayleigh wave dispersion curve from the stacked slowness-frequency (p-f) spectrum.  

- The normalized and stacked power spectrum is displayed as an image, with frequency on the x-axis and apparent velocity on the y-axis.
- The user manually selects points along the visible dispersion curve by clicking on the plot; pressing Enter finishes the selection.
- The picked points are sorted and interpolated using a cubic spline to produce a smooth dispersion curve.
- The result is visualized by overlaying the picked points and the interpolated curve on the power spectrum image.

This process allows for extraction of the fundamental mode Rayleigh wave phase velocity dispersion curve for subsequent 1D shear-wave velocity inversion.

In [12]:
# Get frequency and slowness arrays
freq = sp_fil_chunked_taper_taup_dft[0].get_coord('ft_time')[:summed.shape[1]]
slowness = sp_fil_chunked_taper_taup_dft[0].get_coord('slowness')[half:]

# Create figure
fig, ax = plt.subplots(figsize=(10, 5))
extent = [freq[0], freq[-1], 1/slowness[0], 1/slowness[-1]] # Frequency and apparent velocity extent
im = ax.imshow(
    summed_avg_stack**2,
    aspect='auto',
    origin='lower',
    extent=extent,
    cmap='plasma',
    vmin=0,
    vmax=np.max(summed_avg_stack**2)*0.02  # Adjust for better visibility
)
ax.set_xlabel('Frequency (Hz)', fontsize=14)
ax.set_ylabel('Apparent Velocity (m/s)', fontsize=14)
ax.set_title('Click to pick points, then press Enter', fontsize=14)
ax.set_xlim(1, 20)
ax.set_ylim(100, 600)
fig.colorbar(im, ax=ax, label='Amplitude')

# Pick points on that same figure
print("Click to pick points along the dispersion curve. Press Enter when done.")
picked = fig.ginput(n=-1, timeout=0)  # block until Enter

plt.close(fig)

picked = np.array(picked)
if picked.shape[0] < 2:
    print("Not enough points picked for interpolation.")
else:
    # Sort and interpolate
    picked = picked[np.argsort(picked[:, 0])]
    cs = CubicSpline(picked[:, 0], picked[:, 1])
    freq_interp = np.linspace(picked[:, 0].min(), picked[:, 0].max(), 200)
    velocity_interp = cs(freq_interp)

    # Plot result with cubic spline interpolation
    fig2, ax2 = plt.subplots(figsize=(10, 5))
    im2 = ax2.imshow(
        summed_avg_stack**2,
        aspect='auto',
        origin='lower',
        extent=extent,
        cmap='plasma',
        vmin=0,
        vmax=np.max(summed_avg_stack**2)*0.02 
    )
    ax2.set_xlabel('Frequency (Hz)', fontsize=14)
    ax2.set_ylabel('Apparent Velocity (m/s)', fontsize=14)
    ax2.set_title('Manual Picked Dispersion and Cubic Spline Interpolation', fontsize=14)
    ax2.set_xlim(1, 20)
    ax2.set_ylim(100, 600)
    fig2.colorbar(im2, ax=ax2, label='Amplitude')
    ax2.plot(picked[:, 0], picked[:, 1], 'ro', label='Picked Points')
    ax2.plot(freq_interp, velocity_interp, 'b-', linewidth=2, label='Cubic Spline')
    ax2.legend()
    plt.show()


NameError: name 'summed' is not defined

In [14]:
# Plot static image of the summed power spectrum
# Get frequency and slowness arrays
freq = sp_fil_chunked_taper_taup_dft[0].get_coord('ft_time')[:summed.shape[1]]
slowness = sp_fil_chunked_taper_taup_dft[0].get_coord('slowness')[half:]

plt.figure(figsize=(10, 5))
extent = [freq[0], freq[-1],  1/slowness[0], 1/slowness[-1],]
plt.imshow(
    summed_avg_stack**2,
    aspect='auto',
    origin='lower',
    extent=extent,
    cmap='plasma',
    vmin=0,
    vmax=np.max(summed_avg_stack**2)*0.02
)
plt.plot(picked[:, 0], picked[:, 1], 'ro', label='Picked Points')
plt.plot(freq_interp, velocity_interp, 'b-', linewidth=2, label='Cubic Spline')
plt.xlabel('Frequency (Hz)',fontsize=14)
plt.ylabel('Apparent Velocity (m/s)',fontsize=14)
plt.title('Summed Power Spectrum',fontsize=14)
plt.colorbar(label='Amplitude')
plt.xlim(1,20)
plt.ylim(100,600)
plt.show()

NameError: name 'picked' is not defined

### Correlation-Enhanced ReMi (Ce-ReMi) for DAS

This section implements a fast correlation-domain upgrade to ReMi using either MVDR (Capon) or coherence-weighted Bartlett beamforming on a linear DAS array.

Workflow:
- Windowing + QC (optional lightweight checks)
- Phase-only/whitened normalization per channel in frequency
- Cross-spectral matrix R(f) via Welch-style averaging over accepted windows
- Signed-slowness steering and adaptive power computation
- Frequency stacking to form a sharp f–p (or f–v) image

Notes:
- Efficient: one R(f) inversion per frequency (reused for all slownesses)
- Scales to DAS: distance-decimation and subarray MVDR are supported
- You can keep the same picking workflow (manual or automated)


### Ce-ReMi using existing chunk/map structure (spool-integrated)

This variant reuses your `sp_fil -> chunk -> taper(time) -> taper(distance)` pipeline.
It accumulates cross-spectral matrices across chunks and then performs MVDR/coherence beamforming over a slowness grid.


DASCore Patch ⚡
---------------
➤ Coordinates (distance: 120, time: 2999)
    *distance: CoordRange( min: 5.03e+02 max: 6e+02 step: 0.817 shape: (120,) dtype: float64 units: m )
    *time: CoordRange( min: 2024-08-18T22:58:07.349438464 max: 2024-08-18T22:58:37.329918144 step: 0.01000016s shape: (2999,) dtype: datetime64[ns] units: s )
➤ Data (float64, units: 1.0 / s)
   [[-0.000e+00  0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00  0.000e+00]
    [ 0.000e+00  6.465e-17  2.793e-16 ... -3.180e-12  3.674e-13 -0.000e+00]
    [-0.000e+00 -4.399e-17 -1.998e-16 ...  3.400e-11 -1.371e-11 -0.000e+00]
    ...
    [ 0.000e+00  3.417e-16  1.108e-15 ...  1.698e-12 -1.063e-13  0.000e+00]
    [ 0.000e+00  2.186e-16  7.023e-16 ...  5.980e-14  4.384e-14 -0.000e+00]
    [-0.000e+00 -0.000e+00 -0.000e+00 ... -0.000e+00 -0.000e+00 -0.000e+00]]
➤ Attributes
    data_type: strain_rate
    data_units: 1 / s
    instrument_id: TrebleIISystem04
    history: ("set_units(data_units='1/s',distance='m',time='s')", 

In [8]:
# --- Build cross-spectral matrices R(f) from chunked spool ---
from numpy.fft import rfft, rfftfreq
from scipy.signal import get_window

# Fallback definitions if not imported earlier in this notebook
try:
    compute_window_metrics
except NameError:
    def compute_window_metrics(x):
        pow2 = np.mean(x**2, axis=1) + 1e-12
        pow4 = np.mean(x**4, axis=1)
        krt = np.median(pow4 / (pow2**2 + 1e-18))
        X = np.abs(rfft(x, axis=1)) + 1e-12
        gmean = np.exp(np.mean(np.log(X), axis=1))
        amean = np.mean(X, axis=1) + 1e-12
        flat = np.median(gmean / amean)
        med = np.median(x, axis=0)
        n = med.size
        n_sta = max(1, n // 50)
        n_lta = max(1, n // 5)
        sta = np.convolve(np.abs(med), np.ones(n_sta)/n_sta, mode='same')
        lta = np.convolve(np.abs(med), np.ones(n_lta)/n_lta, mode='same') + 1e-12
        stalta = float(np.nanmax(sta / lta))
        return krt, flat, stalta

try:
    phase_only_normalize
except NameError:
    def phase_only_normalize(X):
        mag = np.abs(X) + 1e-12
        return X / mag

try:
    prewhiten_channels
except NameError:
    def prewhiten_channels(X):
        amp = np.maximum(np.median(np.abs(X), axis=1, keepdims=True), 1e-12)
        return X / amp

# Extract distance/time axes and fs from the already-processed spool
patch0 = sp_fil_chunked_taper_d[0]
dist_axis = np.asarray(patch0.get_coord('distance'))
# DASCore time is datetime64; convert to seconds before computing fs.
time_vals = np.asarray(patch0.get_coord('time')).astype('datetime64[ns]')
# Use the median delta in nanoseconds; convert to seconds
dt_sec = np.median(np.diff(time_vals).astype('timedelta64[ns]').astype(np.int64)) * 1e-9
fs_chunk = 1.0 / float(dt_sec)

# Configure Ce-ReMi parameters (reuse earlier choices if desired)
win_len_s_ce = 20.0
step_s_ce = 10.0
qc_thresholds_ce = (20.0, 0.6, 4.0)

p_min, p_max, p_step = 1/1000.0, 1/100.0, 1/4000.0
slowness_grid_ce = np.arange(p_min, p_max + 1e-9, p_step)

fmin_ce, fmax_ce = 1.0, 30.0
beam_method_ce = 'mvdr'  # 'mvdr' or 'coh'

# Initialize accumulators lazily after first chunk
R_acc = None
n_used_total = 0

for patch in sp_fil_chunked_taper_d:
    # Time-domain data [distance, time]
    X = np.asarray(patch.data)
    if X.shape[0] < X.shape[1]:
        data_chunk = X
    else:
        data_chunk = X.T  # [n_channels, n_samples]

    n_channels, n_samples = data_chunk.shape

    n_win = int(win_len_s_ce * fs_chunk)
    n_step = int(step_s_ce * fs_chunk)
    if n_win <= 8 or n_samples < n_win:
        continue

    tap = get_window('hann', n_win, fftbins=True).astype(float)

    # Slide windows inside this chunk
    for start in range(0, n_samples - n_win + 1, n_step):
        seg = data_chunk[:, start:start+n_win]
        # QC
        krt, flat, stalta = compute_window_metrics(seg)
        krt_max, flat_min, stalta_max = qc_thresholds_ce
        if not (krt < krt_max and flat > flat_min and stalta < stalta_max):
            continue

        seg_w = seg * tap
        Xf = rfft(seg_w, axis=1)  # [n_channels, n_freq]
        Xf = prewhiten_channels(Xf)
        Xf = phase_only_normalize(Xf)

        if R_acc is None:
            n_freq = Xf.shape[1]
            R_acc = np.zeros((n_freq, n_channels, n_channels), dtype=np.complex128)
            freqs_ce = rfftfreq(n_win, d=1.0/fs_chunk)

        if Xf.shape[1] != R_acc.shape[0]:
            # Skip if inconsistent frequency length (shouldn't happen with fixed window)
            continue

        Xf_fcn = np.transpose(Xf, (1,0))  # [n_freq, n_channels]
        R_acc += Xf_fcn[:, :, None] * np.conjugate(Xf_fcn[:, None, :])
        n_used_total += 1

# If no windows passed QC, fall back to using all windows without QC to ensure non-empty output
if n_used_total == 0:
    R_acc = None
    for patch in sp_fil_chunked_taper_d:
        X = np.asarray(patch.data)
        data_chunk = X if X.shape[0] < X.shape[1] else X.T
        n_channels, n_samples = data_chunk.shape
        n_win = int(win_len_s_ce * fs_chunk)
        if n_samples < n_win:
            continue
        tap = get_window('hann', n_win, fftbins=True).astype(float)
        seg = data_chunk[:, :n_win]
        seg_w = seg * tap
        Xf = rfft(seg_w, axis=1)
        Xf = prewhiten_channels(Xf)
        Xf = phase_only_normalize(Xf)
        if R_acc is None:
            n_freq = Xf.shape[1]
            R_acc = np.zeros((n_freq, n_channels, n_channels), dtype=np.complex128)
            freqs_ce = rfftfreq(n_win, d=1.0/fs_chunk)
        Xf_fcn = np.transpose(Xf, (1,0))
        R_acc += Xf_fcn[:, :, None] * np.conjugate(Xf_fcn[:, None, :])
        n_used_total = 1
        break

# Normalize
R_acc = R_acc / max(1, n_used_total)

# Restrict frequency band
fmask_ce = (freqs_ce >= fmin_ce) & (freqs_ce <= fmax_ce)
R_ce = R_acc[fmask_ce]
freqs_sel_ce = freqs_ce[fmask_ce]
print(f"Ce-ReMi: accepted windows total = {n_used_total}, freq bins used = {freqs_sel_ce.size}")


Ce-ReMi: accepted windows total = 100, freq bins used = 580


In [9]:
# --- Beamforming with signed slowness using existing dist_axis ---
# Reuse previously defined beamformer helpers from earlier Ce-ReMi section if present.

# If helpers are not yet defined in this session, define minimal versions here.
try:
    steering_vectors
except NameError:
    def steering_vectors(distance_axis_m, freqs, slowness_s_per_m, sign=+1):
        x = distance_axis_m[None, None, :]
        f = freqs[:, None, None]
        p = (sign * slowness_s_per_m[None, :, None])
        phase = -2j * np.pi * f * p * x
        return np.exp(phase)

try:
    mvdr_power
except NameError:
    def mvdr_power(Rf, A):
        N = Rf.shape[0]
        Rr = Rf + 1e-3 * np.trace(Rf).real / N * np.eye(N)
        try:
            RiA = np.linalg.solve(Rr, A.T).T
        except np.linalg.LinAlgError:
            RiA = A
        denom = np.sum(np.conjugate(A) * RiA, axis=1).real + 1e-18
        return 1.0 / denom

try:
    coherence_weighted_bartlett
except NameError:
    def coherence_weighted_bartlett(Rf, A):
        d = np.sqrt(np.clip(np.real(np.diag(Rf)), 1e-12, None))
        Dinv = 1.0 / d
        Rcorr = (Rf * Dinv[None, :]) * Dinv[:, None]
        return np.einsum('pn,nm,pm->p', np.conjugate(A), Rcorr, A).real


def compute_fp_spectrum_ce(R, freqs, distance_axis_m, slowness_grid, method='mvdr', sign=+1):
    F, N, _ = R.shape
    P = slowness_grid.size
    S = np.zeros((F, P), dtype=float)
    A_all = steering_vectors(distance_axis_m, freqs, slowness_grid, sign=sign)
    for k in range(F):
        Rf = R[k]
        A = A_all[k]
        if method == 'mvdr':
            S[k] = mvdr_power(Rf, A)
        else:
            S[k] = coherence_weighted_bartlett(Rf, A)
    return S

Spos_ce = compute_fp_spectrum_ce(R_ce, freqs_sel_ce, dist_axis, slowness_grid_ce, method=beam_method_ce, sign=+1)
Sneg_ce = compute_fp_spectrum_ce(R_ce, freqs_sel_ce, dist_axis, slowness_grid_ce, method=beam_method_ce, sign=-1)

Sfp_ce = Spos_ce + Sneg_ce
# Protect against degenerate rows
row_mean = np.mean(Sfp_ce, axis=1, keepdims=True)
row_mean[row_mean == 0] = 1.0
Sfp_ce_norm = Sfp_ce / row_mean

# Package outputs for downstream use
ce_out = {
    'freqs': freqs_sel_ce,
    'slowness': slowness_grid_ce,
    'velocity': 1.0 / slowness_grid_ce,
    'power_pf': Sfp_ce_norm,  # [F, P]
    'power_pf_pos': Spos_ce,
    'power_pf_neg': Sneg_ce,
    'R': R_ce,
}

# Visualization
vel_grid_ce = 1.0 / slowness_grid_ce
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

im0 = ax[0].imshow(
    Sfp_ce_norm,
    aspect='auto', origin='lower',
    extent=[slowness_grid_ce[0], slowness_grid_ce[-1], freqs_sel_ce[0], freqs_sel_ce[-1]],
    cmap='plasma'
)
ax[0].set_xlabel('Slowness p (s/m)')
ax[0].set_ylabel('Frequency (Hz)')
ax[0].set_title(f'Ce-ReMi (chunked) {beam_method_ce.upper()} f–p')
fig.colorbar(im0, ax=ax[0])

im1 = ax[1].imshow(
    Sfp_ce_norm,
    aspect='auto', origin='lower',
    extent=[vel_grid_ce[-1], vel_grid_ce[0], freqs_sel_ce[0], freqs_sel_ce[-1]],
    cmap='plasma'
)
ax[1].set_xlabel('Velocity v (m/s)')
ax[1].set_ylabel('Frequency (Hz)')
ax[1].set_title('Ce-ReMi (chunked) f–v')
fig.colorbar(im1, ax=ax[1])
plt.tight_layout(); plt.show()


In [19]:
# --- Dispersion image: velocity (y) vs frequency (x) ---
import numpy as np
import matplotlib.pyplot as plt

# Prefer ce_out if available
if 'ce_out' in globals() and isinstance(ce_out, dict) and 'power_pf' in ce_out:
    Sfp = ce_out['power_pf']           # [F, P]
    freqs = np.asarray(ce_out['freqs'])
    vel = np.asarray(ce_out['velocity'])
else:
    # Fallback to local variables from Ce-ReMi section
    Sfp = Sfp_ce_norm                  # [F, P]
    freqs = np.asarray(freqs_sel_ce)
    vel = 1.0 / np.asarray(slowness_grid_ce)

# Ensure frequency increases left->right
if freqs[0] > freqs[-1]:
    freqs = freqs[::-1]
    Sfp = Sfp[::-1, :]

# Map to velocity on y-axis explicitly; no rotation, just transpose to [P, F]
S_vy = Sfp.T                           # [P, F]
vel_min, vel_max = float(np.min(vel)), float(np.max(vel))

plt.figure(figsize=(9, 5))
im = plt.imshow(
    S_vy,
    aspect='auto',
    origin='lower',
    extent=[float(freqs[0]), float(freqs[-1]), vel_min, vel_max],
    cmap='plasma'
)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Apparent Velocity (m/s)')
plt.title('Ce-ReMi dispersion image (Velocity vs Frequency)')
plt.colorbar(im, label='Normalized power')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [20]:
# --- Comparison: Original ReMi vs Ce-ReMi (Velocity vs Frequency) ---
import numpy as np
import matplotlib.pyplot as plt

# Build Original ReMi arrays
try:
    # Frequencies and slowness from tau-p DFT patch
    patch_dft0 = sp_fil_chunked_taper_taup_dft[0]
    ft_all = np.asarray(patch_dft0.get_coord('ft_time'))
    sl_all = np.asarray(patch_dft0.get_coord('slowness'))
    F_re, P_re = summed_avg_stack.shape  # [freq, slowness]
    f_re = ft_all[:F_re]
    # Follow original indexing convention for positive slowness
    half_local = patch_dft0.data.shape[0] // 2
    sl_pos = sl_all[half_local:half_local + P_re]
    v_re = 1.0 / sl_pos
    S_re = np.asarray(summed_avg_stack)
    # Orient to [velocity, frequency]
    S_re_vy = S_re.T  # [P, F]
except Exception as e:
    raise RuntimeError(f"Original ReMi products not available: {e}")

# Build Ce-ReMi arrays
if 'ce_out' in globals() and isinstance(ce_out, dict) and 'power_pf' in ce_out:
    S_ce = np.asarray(ce_out['power_pf'])   # [F,P]
    f_ce = np.asarray(ce_out['freqs'])
    v_ce = np.asarray(ce_out['velocity'])
    S_ce_vy = S_ce.T                        # [P,F]
else:
    # Fallback to local variables from Ce-ReMi section
    S_ce = np.asarray(Sfp_ce_norm)
    f_ce = np.asarray(freqs_sel_ce)
    v_ce = 1.0 / np.asarray(slowness_grid_ce)
    S_ce_vy = S_ce.T

# Ensure frequency increases
if f_re[0] > f_re[-1]:
    f_re = f_re[::-1]
    S_re_vy = S_re_vy[:, ::-1]
if f_ce[0] > f_ce[-1]:
    f_ce = f_ce[::-1]
    S_ce_vy = S_ce_vy[:, ::-1]

# Common color scale (robust): 99th percentile across both
vmax = np.nanmax([np.percentile(S_re_vy, 99), np.percentile(S_ce_vy, 99)])

fig, ax = plt.subplots(1, 2, figsize=(13, 5), sharey=False)

im0 = ax[0].imshow(
    S_re_vy,
    aspect='auto', origin='lower',
    extent=[float(f_re[0]), float(f_re[-1]), float(v_re[0]), float(v_re[-1])],
    cmap='plasma', vmin=0, vmax=float(vmax)
)
ax[0].set_xlabel('Frequency (Hz)')
ax[0].set_ylabel('Apparent Velocity (m/s)')
ax[0].set_title('Original ReMi')
ax[0].invert_yaxis()
fig.colorbar(im0, ax=ax[0], label='Norm. power')

im1 = ax[1].imshow(
    S_ce_vy,
    aspect='auto', origin='lower',
    extent=[float(f_ce[0]), float(f_ce[-1]), float(v_ce[0]), float(v_ce[-1])],
    cmap='plasma', vmin=0, vmax=float(vmax)
)
ax[1].set_xlabel('Frequency (Hz)')
ax[1].set_ylabel('Apparent Velocity (m/s)')
ax[1].set_title('Ce-ReMi (adaptive)')
ax[1].invert_yaxis()
fig.colorbar(im1, ax=ax[1], label='Norm. power')

plt.tight_layout()
plt.show()

# Simple quantitative comparison: ridge contrast (peak/median) per frequency
with np.errstate(divide='ignore', invalid='ignore'):
    rc_re = np.nanmean(np.nanmax(S_re, axis=1) / (np.nanmedian(S_re, axis=1) + 1e-12))
    rc_ce = np.nanmean(np.nanmax(S_ce, axis=1) / (np.nanmedian(S_ce, axis=1) + 1e-12))
print(f"Ridge contrast (mean across freq): ReMi={rc_re:.2f}, Ce-ReMi={rc_ce:.2f}")


Ridge contrast (mean across freq): ReMi=2.60, Ce-ReMi=1.64


### Synthetic demonstration: Original ReMi vs Correlation-Enhanced ReMi (Ce-ReMi)

This section synthesizes linear-array data with a known dispersion curve and non-stationary interference, then compares:
- Original ReMi (Bartlett beamforming over slowness)
- Ce-ReMi (correlation-domain MVDR/coherence)

Outputs:
- Velocity-vs-frequency images for both methods
- Simple ridge-contrast metric to quantify improvement


In [28]:
v_of_f

NameError: name 'v_of_f' is not defined

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import rfft, irfft, rfftfreq

# --- 1) Synthetic data generator (coherent dispersive packet) ---
def synth_linear_array(
    num_channels=201, aperture_m=200.0, fs=100.0, dur_s=600.0,
    vmin=200.0, vmax=1000.0, fmin=1.0, fmax=20.0,
    noise_std=0.05, movers=False, mover_amp=0.5, random_seed=0,
    coherent=True, t0=10.0, fc=10.0, bandwidth_hz=12.0,
):
    """
    Build a coherent, dispersive surface-wave packet so dispersion is visible in time–distance.
    coherent=True uses a Gaussian band around fc with a common time shift t0 and
    phase steering exp(-i 2π f p(f) x), producing a visible dispersive wavefront.
    """
    rng = np.random.default_rng(random_seed)
    x = np.linspace(0, aperture_m, num_channels)
    t = np.arange(0, dur_s, 1.0/fs)
    F = rfftfreq(t.size, d=1/fs)

    # Dispersion law v(f): linear between (fmin,vmax) and (fmax,vmin)
    v_of_f = np.interp(F, [fmin, fmax], [vmax, vmin])
    p_of_f = 1.0 / np.maximum(v_of_f, 1e-6)

    # Frequency-domain source amplitude
    if coherent:
        sigma = bandwidth_hz / 2.355  # FWHM -> sigma
        Amp = np.exp(-0.5 * ((F - fc) / max(sigma, 1e-6))**2)
        Amp *= (F >= fmin) & (F <= fmax)
        phase_src = np.exp(-2j * np.pi * F * t0)  # center time t0
    else:
        Amp = ((F >= fmin) & (F <= fmax)).astype(float)
        phase_src = np.exp(1j * rng.uniform(0, 2*np.pi, size=F.size))

    # Build channel spectra with dispersive steering
    Xf = np.zeros((num_channels, F.size), dtype=np.complex128)
    for n, xn in enumerate(x):
        steer = np.exp(-2j * np.pi * F * p_of_f * xn)
        Xf[n] = Amp * phase_src * steer

    sfc = irfft(Xf, axis=1)

    # Add white noise
    sfc += noise_std * rng.standard_normal(sfc.shape)

    # Optional non-stationary movers
    if movers:
        n_movers = 2
        for k in range(n_movers):
            t0m = rng.uniform(5, dur_s-5)
            v_move = rng.uniform(200, 1000)
            f_move = rng.uniform(5, 15)
            for n, xn in enumerate(x):
                tau = (xn / v_move)
                w = np.exp(-0.5 * ((t - (t0m + tau))/1.0)**2)
                sfc[n] += mover_amp * w * np.sin(2*np.pi*f_move*(t - (t0m + tau)))

    return x, t, sfc

# --- 2) Baseline ReMi (Bartlett) ---
def remi_bartlett_power(sfc, x, fs, slowness_grid, fmin, fmax):
    # FFT per channel
    X = rfft(sfc, axis=1)
    F = rfftfreq(sfc.shape[1], d=1/fs)
    fmask = (F >= fmin) & (F <= fmax)
    X = X[:, fmask]
    Fsel = F[fmask]

    # Steering and Bartlett power
    P = slowness_grid.size
    S = np.zeros((Fsel.size, P))
    for k, fk in enumerate(Fsel):
        a = np.exp(-2j*np.pi*fk*slowness_grid[None,:]*x[:,None])  # [N,P]
        S[k] = np.abs(np.einsum('n,np->p', X[:,k], np.conjugate(a)))**2
    # Normalize per-frequency
    S /= (np.mean(S, axis=1, keepdims=True) + 1e-12)
    return Fsel, S

# --- 3) Ce-ReMi (correlation MVDR) ---
def cermi_mvdr_power(sfc, x, fs, slowness_grid, fmin, fmax):
    from numpy.linalg import LinAlgError

    X = rfft(sfc, axis=1)
    F = rfftfreq(sfc.shape[1], d=1/fs)
    fmask = (F >= fmin) & (F <= fmax)
    X = X[:, fmask]
    Fsel = F[fmask]

    # Phase-only normalization
    X = X / (np.abs(X) + 1e-12)

    # CSM
    R = np.einsum('nf,mf->fmn', X, np.conjugate(X)) / X.shape[1]

    # MVDR per frequency
    P = slowness_grid.size
    S = np.zeros((Fsel.size, P))
    for k, fk in enumerate(Fsel):
        Rf = R[k]
        N = Rf.shape[0]
        Rr = Rf + 1e-3 * np.trace(Rf).real / max(N,1) * np.eye(N)
        a = np.exp(-2j*np.pi*fk*slowness_grid[None,:]*x[:,None])  # [N,P]
        try:
            RiA = np.linalg.solve(Rr, a)
        except LinAlgError:
            RiA = a
        denom = np.sum(np.conjugate(a) * RiA, axis=0).real + 1e-18
        S[k] = 1.0 / denom
    S /= (np.mean(S, axis=1, keepdims=True) + 1e-12)
    return Fsel, S

# --- 4) Run experiment ---
x, t, sfc = synth_linear_array(coherent=True, movers=False, noise_std=0.05, fc=10.0, bandwidth_hz=12.0)
fs_syn = 1.0 / np.median(np.diff(t))

p_grid = np.linspace(1/800.0, 1/120.0, 161)
fmin, fmax = 2.0, 20.0

F_re, S_re = remi_bartlett_power(sfc, x, fs_syn, p_grid, fmin, fmax)
F_ce, S_ce = cermi_mvdr_power(sfc, x, fs_syn, p_grid, fmin, fmax)

V = 1.0 / p_grid

# Prepare images as [velocity, frequency]
S_re_vy = S_re.T
S_ce_vy = S_ce.T

# Common color scale
vmax = np.nanmax([np.percentile(S_re_vy, 99), np.percentile(S_ce_vy, 99)])

fig, ax = plt.subplots(1, 2, figsize=(13, 5))
im0 = ax[0].imshow(S_re_vy, aspect='auto', origin='lower',
                   extent=[F_re[0], F_re[-1], V.min(), V.max()], cmap='plasma', vmin=0, vmax=vmax)
ax[0].set_title('Synthetic - Original ReMi')
ax[0].set_xlabel('Frequency (Hz)'); ax[0].set_ylabel('Velocity (m/s)'); ax[0].invert_yaxis()
fig.colorbar(im0, ax=ax[0])

im1 = ax[1].imshow(S_ce_vy, aspect='auto', origin='lower',
                   extent=[F_ce[0], F_ce[-1], V.min(), V.max()], cmap='plasma', vmin=0, vmax=vmax)
ax[1].set_title('Synthetic - Ce-ReMi (MVDR)')
ax[1].set_xlabel('Frequency (Hz)'); ax[1].set_ylabel('Velocity (m/s)'); ax[1].invert_yaxis()
fig.colorbar(im1, ax=ax[1])
plt.tight_layout(); plt.show()

# Contrast metric
rc_re = np.nanmean(np.nanmax(S_re, axis=1) / (np.nanmedian(S_re, axis=1) + 1e-12))
rc_ce = np.nanmean(np.nanmax(S_ce, axis=1) / (np.nanmedian(S_ce, axis=1) + 1e-12))
print(f"Synthetic ridge contrast: ReMi={rc_re:.2f}, Ce-ReMi={rc_ce:.2f}")


Synthetic ridge contrast: ReMi=5.92, Ce-ReMi=1.02


In [32]:
# --- Visualize synthetic time–distance data ---
import numpy as np
import matplotlib.pyplot as plt

# Ensure synthetic variables exist (run the synthetic cell first)
assert 'x' in globals() and 't' in globals() and 'sfc' in globals(), "Run the synthetic generation cell first."

# 1) Waterfall image (distance vs time)
vmax = np.percentile(np.abs(sfc), 99)
plt.figure(figsize=(10, 4))
plt.imshow(
    sfc,
    aspect='auto',
    origin='lower',
    extent=[float(t[0]), float(t[-1]), float(x[0]), float(x[-1])],
    cmap='seismic', vmin=-vmax, vmax=vmax,
)
plt.xlabel('Time (s)')
plt.ylabel('Distance (m)')
plt.title('Synthetic waterfall (time vs distance)')
plt.tight_layout(); plt.show()

# 2) Selected normalized traces (stacked)
fig, ax = plt.subplots(figsize=(10, 5))
num_show = 8
chan_idx = np.linspace(0, sfc.shape[0] - 1, num_show, dtype=int)
for k, ci in enumerate(chan_idx):
    tr = sfc[ci]
    tr = tr / (np.max(np.abs(tr)) + 1e-12)
    ax.plot(t, tr + k * 1.6, lw=0.8, label=f'x={x[ci]:.0f} m')
ax.set_xlabel('Time (s)')
ax.set_yticks([])
ax.set_title('Selected normalized traces (offset for visibility)')
ax.legend(ncol=2, fontsize=8, frameon=False)
plt.tight_layout(); plt.show()

# 3) Spectrogram of a mid-channel
mid = sfc.shape[0] // 2
fig, ax = plt.subplots(figsize=(10, 4))
Pxx, f_spec, t_spec, im = ax.specgram(sfc[mid], NFFT=256, Fs=float(1.0/np.median(np.diff(t))), noverlap=128, cmap='magma')
ax.set_ylim(0, 40)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (Hz)')
ax.set_title(f'Spectrogram (mid-channel at x={x[mid]:.0f} m)')
plt.tight_layout(); plt.show()


In [ ]:
sfc

(64, 6000)